# CIS 678 - Machine Learning - Kaggle Challenge 3: MNIST Digit Classification

## Data Loading

## Model Implementation

## Model Evaluation

## Conclusion

In [1]:
!echo sourcing MNIST model...
%cd '../model/src/'

sourcing MNIST model...


/Users/jppalacios/academic/fall2023/cis678/ml-challenge-3/model/src


In [2]:
import numpy as np

from model import Model

model = Model()

model.configure(debug_mode = True)

MNIST model logger - DEBUG - running in debug mode!
